# Ephemerides scraper

## 1: list of links to each year

In [15]:
# based on https://github.com/justmarkham/trump-lies/blob/master/trump_lies.ipynb

import requests
r = requests.get('https://www.findyourfate.com/astrology/ephemeris/ephemeris.html')

In [17]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, 'html.parser')

In [24]:
cells = soup.find_all('div', attrs={'class':'Cell'})

In [25]:
cells[0]

<div class="Cell" style="padding-left:5px; padding-right:5px; padding-top:5px; padding-bottom:5px;">
<a href="https://www.findyourfate.com/astrology/ephemeris/1900.html">Ephemeris for year 1900</a></div>

In [31]:
links = []
for cell in cells:
  year = cell.find('a').text[-4:]
  href = cell.find('a')['href']
  links.append((year, href))

## 2: angles for each year

In [118]:
import json

cut_off = 2100

ephemerides = {}

for link in links:
  href = link[1]
  year = link[0]
  
  if int(year) < cut_off:
    continue
  
  r = requests.get(href)
  soup = BeautifulSoup(r.text, 'html.parser')
  months = soup.find_all('pre')
  
  ephemerides[year] = {}
  
  print(f"Starting {year}...")

  for i, month in enumerate(months):
    i_month = str(i+1).zfill(2)
    
    
#     print(f"Starting month: {i_month}...")

    ephemerides[year][i_month] = {}
    
    aux = month.find_all('b')
    
    # the first <b> is the header with the planets, which we don't need.
    # [::2] comes from https://stackoverflow.com/questions/28883769/remove-odd-indexed-elements-from-list-in-python
    # and is as follows: a[start:end:step] --> start through not past end, by step
    days = aux[len(aux)-1].text.splitlines()[::2]
    
    for day in days:
      aux = day.strip().split(' ')
      
      # print(f"...has {len(aux)} element(s)")
      if len(aux) >= 14:
        # DATE  SID.TIME  SUN   MOON   MERCURY  VENUS  MARS JUPITER SATURN URANUS NEPTUNE PLUTO  NODE
        data = {
          'time': aux[2],
          'sun': aux[3],
          'moon': aux[4],
          'mercury': aux[5],
          'venus': aux[6],
          'mars': aux[7],
          'jupiter': aux[8],
          'saturn': aux[9],
          'uranus': aux[10],
          'neptune': aux[11],
          'pluto': aux[12],
          'node': aux[13]
        }
        ephemerides[year][i_month][aux[1]] = data
        
  fn = 'raw_data/' + year + '.json'
  with open(fn, 'w') as fp:
    json.dump(ephemerides[year], fp, sort_keys=True, indent=2)
    

Starting 2100...
